In [99]:
import numpy as np
from numpy.core.fromnumeric import _all_dispatcher
import pandas as pd
import joblib
np.random.seed(2021)
import warnings
warnings.filterwarnings('ignore')

In [70]:
##########################################################
######### training code (without any validation) #########

# load data
print('loading train.gz...')
# use only a subset of rows - you should use all rows eventually
df_train = pd.read_csv("train.gz", compression='gzip', nrows=20000, header='infer')
Y = df_train['click']
# discard some columns
unused_cols = ["id", 'site_id', 'app_id']
df_train.drop(unused_cols, axis=1, inplace=True)

loading train.gz...


In [71]:
# create a copy to prevent from modifying the original dataset
df_copy = df_train.copy()

In [72]:
# nunique() shows that the three columns are same across 
df_copy.drop(['hour', 'app_domain', 'app_category', 'click'], axis=1,inplace=True)

In [73]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(df_copy, Y, test_size=0.3)

In [ ]:
X_train.hour

In [74]:
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# one_hot_features = ['C1', 'device_type', 'device_conn_type', 'C18']
one_hot_features = [0,7,8,13]
one_hot_transformer = OneHotEncoder(drop='first')

# target_features = ['banner_pos','site_domain','site_category','device_id','device_ip','device_model','C14',
#                    'C15','C16','C17','C19','C20','C21']
target_features = [1,2,3,4,5,6,9,10,11,12,14,15,16]
target_transformer = ce.JamesSteinEncoder()
        
preprocessor = ColumnTransformer(
    transformers=[
        ('one_hot', one_hot_transformer, one_hot_features),
        ('target', target_transformer, target_features)])

In [75]:
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostClassifier(iterations=20,learning_rate=0.1,depth=7, eval_metric='Logloss'))])

clf.fit(X_train.values, y_train.values.reshape(-1,1))
y_pred = clf.predict_proba(X_test.values)[:, 1]
print("model logloss: %.3f" % log_loss(y_test, y_pred))

/Users/williamlee/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0:	learn: 0.6180901	total: 32.1ms	remaining: 611ms
1:	learn: 0.5544265	total: 36.6ms	remaining: 329ms
2:	learn: 0.5013944	total: 41.4ms	remaining: 235ms
3:	learn: 0.4562998	total: 45.3ms	remaining: 181ms
4:	learn: 0.4178803	total: 47.7ms	remaining: 143ms
5:	learn: 0.3851783	total: 51.1ms	remaining: 119ms
6:	learn: 0.3563879	total: 59.4ms	remaining: 110ms
7:	learn: 0.3316121	total: 62.8ms	remaining: 94.1ms
8:	learn: 0.3101531	total: 67.7ms	remaining: 82.7ms
9:	learn: 0.2907200	total: 72ms	remaining: 72ms
10:	learn: 0.2750967	total: 76.1ms	remaining: 62.3ms
11:	learn: 0.2602556	total: 80.4ms	remaining: 53.6ms
12:	learn: 0.2482299	total: 86.3ms	remaining: 46.5ms
13:	learn: 0.2367818	total: 90.8ms	remaining: 38.9ms
14:	learn: 0.2264693	total: 94.3ms	remaining: 31.4ms
15:	learn: 0.2171684	total: 100ms	remaining: 25ms
16:	learn: 0.2083740	total: 105ms	remaining: 18.5ms
17:	learn: 0.2015494	total: 109ms	remaining: 12.2ms
18:	learn: 0.1947537	total: 113ms	remaining: 5.95ms
19:	learn: 0.1883837

#### First baseline model has a very high logloss at 0.525.

In [76]:
categorical_f = ['C1','device_type','device_conn_type','C18','banner_pos','site_domain','site_category',
                 'device_id','device_ip','device_model','C14','C15','C16','C17','C19','C20','C21']
cat = CatBoostClassifier(iterations=20,learning_rate=0.1,depth=7, eval_metric='Logloss')
cat.fit(X_train, y_train,cat_features=categorical_f)
y_pred_cat = cat.predict_proba(X_test.values)[:, 1]
print("model logloss: %.3f" % log_loss(y_test, y_pred_cat))

0:	learn: 0.6548731	total: 22.6ms	remaining: 429ms
1:	learn: 0.6253654	total: 30.9ms	remaining: 278ms
2:	learn: 0.5983279	total: 40.4ms	remaining: 229ms
3:	learn: 0.5760107	total: 54ms	remaining: 216ms
4:	learn: 0.5575109	total: 62.1ms	remaining: 186ms
5:	learn: 0.5417719	total: 70.8ms	remaining: 165ms
6:	learn: 0.5286705	total: 80.2ms	remaining: 149ms
7:	learn: 0.5174565	total: 89.1ms	remaining: 134ms
8:	learn: 0.5079688	total: 94.1ms	remaining: 115ms
9:	learn: 0.5000841	total: 103ms	remaining: 103ms
10:	learn: 0.4931135	total: 112ms	remaining: 91.3ms
11:	learn: 0.4868609	total: 123ms	remaining: 81.7ms
12:	learn: 0.4825855	total: 125ms	remaining: 67.5ms
13:	learn: 0.4782327	total: 133ms	remaining: 57.2ms
14:	learn: 0.4742199	total: 143ms	remaining: 47.5ms
15:	learn: 0.4709195	total: 147ms	remaining: 36.8ms
16:	learn: 0.4679939	total: 157ms	remaining: 27.7ms
17:	learn: 0.4656034	total: 160ms	remaining: 17.8ms
18:	learn: 0.4635120	total: 168ms	remaining: 8.86ms
19:	learn: 0.4620704	tota

#### using the default catboost encoder seems to get much better result

### Bayesian Search CV for hyperparameter tuning

In [45]:
# need to balance the data
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, RandomOverSampler
# smt = SMOTE()
ros = RandomOverSampler(random_state=0)
X_train_re, y_train_re = ros.fit_resample(X_train, y_train)

In [46]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer

cat1 = CatBoostClassifier(iterations=20,learning_rate=0.1,depth=7,loss_function='Logloss', 
                          cat_features=categorical_f,verbose=False)

param = {
    'iterations': Integer(10, 1000),
    'depth': Integer(1, 8),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'random_strength': Real(1e-9, 10, 'log-uniform'),
    'bagging_temperature': Real(0.0, 1.0),
    'border_count': Integer(1, 255),
    'l2_leaf_reg': Integer(2, 30),
    'scale_pos_weight':Real(0.01, 1.0, 'uniform')
}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    cat1,
    param,
    scoring = LogLoss,
    n_iter=64,
    cv=5,
    random_state=42
)

# executes bayesian optimization
opt.fit(X_train_re, y_train_re)

BayesSearchCV(cv=5,
              estimator=<catboost.core.CatBoostClassifier object at 0x7fa17b3d6130>,
              n_iter=64, random_state=42,
              scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
              search_spaces={'bagging_temperature': Real(low=0.0, high=1.0, prior='uniform', transform='identity'),
                             'border_count': Integer(low=1, high=255, prior='uniform', transform='ident...
                             'iterations': Integer(low=10, high=1000, prior='uniform', transform='identity'),
                             'l2_leaf_reg': Integer(low=2, high=30, prior='uniform', transform='identity'),
                             'learning_rate': Real(low=0.01, high=1.0, prior='log-uniform', transform='identity'),
                             'random_strength': Real(low=1e-09, high=10, prior='log-uniform', transform='identity'),
                             'scale_pos_weight': Real(low=0.01, high=1.0, prior='uniform', t

In [48]:
opt.best_score_

-0.1523188638805765

In [100]:
df_xgb = df_copy.copy()
def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self
df_xgb = convert_obj_to_int(df_xgb)

In [101]:
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(df_xgb, Y, test_size=0.3)

In [102]:
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3,7),
    'subsample': (1.0, 1.0),  # Change for big datasets
    'colsample': (1.0, 1.0),  # Change for datasets with lots of features
    'gamma': (0, 10)}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

def xgboost_hyper_param(learning_rate,n_estimators,max_depth,subsample,colsample,gamma):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    clf = XGBClassifier(max_depth=max_depth,learning_rate=learning_rate,
                        n_estimators=n_estimators,gamma=gamma)
    
    return np.mean(cross_val_score(clf, X_train_xgb, y_train_xgb, cv=5, scoring=LogLoss))

optimizer = BayesianOptimization(f=xgboost_hyper_param, pbounds=pbounds)

In [103]:
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | colsample |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
[17:40:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:40:14] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:40:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric i

[17:41:39] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:41:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:41:46] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:41:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[17:42:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:42:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  11       | -0.4515   |  1.0      |  4.488    |  0.1349   |  4.726    |  140.4    |  1.0      |
[17:42:29] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:42:32] WARNING: /Users/travis/build/dmlc/xgboost/src

|  16       | -0.4507   |  1.0      |  0.213    |  0.02439  |  4.078    |  917.3    |  1.0      |
[17:43:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:43:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:43:30] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:43:32] WARNING: /Users/travis/build/dmlc/xgboost/src

[17:44:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:11] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

...

[17:46:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:46:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:46:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:46:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[17:47:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:47:28] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  43       | -0.5783   |  1.0      |  0.2911   |  0.6635   |  6.545    |  553.0    |  1.0      |
[17:47:32] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:47:33] WARNING: /Users/travis/build/dmlc/xgboost/src

|  48       | -0.4538   |  1.0      |  9.578    |  0.3843   |  3.097    |  324.6    |  1.0      |
[17:48:14] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:48:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:48:17] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:48:18] WARNING: /Users/travis/build/dmlc/xgboost/src

[17:49:16] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:49:19] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:49:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:49:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[17:50:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:50:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  59       | -0.4854   |  1.0      |  0.1283   |  0.6028   |  3.934    |  250.4    |  1.0      |
[17:50:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:50:27] WARNING: /Users/travis/build/dmlc/xgboost/src

In [105]:
optimizer.max

{'target': -0.45048454820444545,
 'params': {'colsample': 1.0,
  'gamma': 4.4036150622160655,
  'learning_rate': 0.39547097261826947,
  'max_depth': 5.521033605627301,
  'n_estimators': 141.9601037125904,
  'subsample': 1.0}}

In [54]:
from sklearn.metrics import make_scorer
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

best_params={'bagging_temperature': 0.0,
 'border_count': 48,
 'depth': 8,
 'iterations': 759,
 'l2_leaf_reg': 3,
 'learning_rate': 0.01,
 'random_strength': 1e-09,
 'scale_pos_weight': 1.0}

# best_params['iterations'] = 1000

In [50]:
opt_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostClassifier(**best_params,od_type='Iter', loss_function='Logloss'))])

from sklearn.model_selection import cross_val_score
log_ = cross_val_score(estimator = opt_clf, X = X_train_re.values, y = y_train_re.values.reshape(-1,1), 
                             cv = 5, scoring=LogLoss)

/Users/williamlee/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0:	learn: 0.1952359	total: 6.73ms	remaining: 6.72s
1:	learn: 0.1895334	total: 14.2ms	remaining: 7.11s
2:	learn: 0.1887291	total: 20.4ms	remaining: 6.76s
3:	learn: 0.1882983	total: 26.5ms	remaining: 6.61s
4:	learn: 0.1700176	total: 32.8ms	remaining: 6.52s
5:	learn: 0.1693073	total: 38.7ms	remaining: 6.42s
6:	learn: 0.1678611	total: 44.9ms	remaining: 6.36s
7:	learn: 0.1672434	total: 51.2ms	remaining: 6.34s
8:	learn: 0.1649212	total: 58.4ms	remaining: 6.43s
9:	learn: 0.1626958	total: 68.5ms	remaining: 6.78s
10:	learn: 0.1617119	total: 75ms	remaining: 6.75s
11:	learn: 0.1604154	total: 81.3ms	remaining: 6.69s
12:	learn: 0.1583953	total: 87.5ms	remaining: 6.64s
13:	learn: 0.1533844	total: 93.5ms	remaining: 6.58s
14:	learn: 0.1516842	total: 99.1ms	remaining: 6.5s
15:	learn: 0.1507243	total: 105ms	remaining: 6.47s
16:	learn: 0.1495447	total: 111ms	remaining: 6.43s
17:	learn: 0.1485590	total: 117ms	remaining: 6.4s
18:	learn: 0.1479176	total: 124ms	remaining: 6.38s
19:	learn: 0.1473076	total: 12

162:	learn: 0.1049265	total: 1.2s	remaining: 6.17s
163:	learn: 0.1048240	total: 1.22s	remaining: 6.21s
164:	learn: 0.1044531	total: 1.24s	remaining: 6.26s
165:	learn: 0.1043234	total: 1.25s	remaining: 6.3s
166:	learn: 0.1042118	total: 1.27s	remaining: 6.34s
167:	learn: 0.1040699	total: 1.28s	remaining: 6.36s
168:	learn: 0.1038910	total: 1.29s	remaining: 6.35s
169:	learn: 0.1036909	total: 1.3s	remaining: 6.34s
170:	learn: 0.1034522	total: 1.3s	remaining: 6.32s
171:	learn: 0.1032589	total: 1.31s	remaining: 6.31s
172:	learn: 0.1028909	total: 1.32s	remaining: 6.29s
173:	learn: 0.1026151	total: 1.32s	remaining: 6.28s
174:	learn: 0.1023135	total: 1.33s	remaining: 6.27s
175:	learn: 0.1021617	total: 1.33s	remaining: 6.25s
176:	learn: 0.1020234	total: 1.34s	remaining: 6.24s
177:	learn: 0.1019950	total: 1.35s	remaining: 6.22s
178:	learn: 0.1018942	total: 1.35s	remaining: 6.21s
179:	learn: 0.1017929	total: 1.36s	remaining: 6.2s
180:	learn: 0.1015954	total: 1.37s	remaining: 6.19s
181:	learn: 0.101

347:	learn: 0.0847334	total: 2.58s	remaining: 4.84s
348:	learn: 0.0847333	total: 2.59s	remaining: 4.83s
349:	learn: 0.0847333	total: 2.59s	remaining: 4.81s
350:	learn: 0.0847333	total: 2.6s	remaining: 4.8s
351:	learn: 0.0847332	total: 2.6s	remaining: 4.79s
352:	learn: 0.0847331	total: 2.6s	remaining: 4.77s
353:	learn: 0.0847331	total: 2.61s	remaining: 4.76s
354:	learn: 0.0847331	total: 2.61s	remaining: 4.74s
355:	learn: 0.0847330	total: 2.61s	remaining: 4.73s
356:	learn: 0.0847330	total: 2.62s	remaining: 4.71s
357:	learn: 0.0847329	total: 2.62s	remaining: 4.7s
358:	learn: 0.0847327	total: 2.62s	remaining: 4.68s
359:	learn: 0.0847327	total: 2.63s	remaining: 4.67s
360:	learn: 0.0847326	total: 2.63s	remaining: 4.66s
361:	learn: 0.0847326	total: 2.63s	remaining: 4.64s
362:	learn: 0.0847325	total: 2.64s	remaining: 4.63s
363:	learn: 0.0847324	total: 2.65s	remaining: 4.63s
364:	learn: 0.0847323	total: 2.66s	remaining: 4.63s
365:	learn: 0.0847323	total: 2.67s	remaining: 4.62s
366:	learn: 0.084

522:	learn: 0.0798890	total: 3.56s	remaining: 3.25s
523:	learn: 0.0798014	total: 3.57s	remaining: 3.25s
524:	learn: 0.0798010	total: 3.58s	remaining: 3.24s
525:	learn: 0.0797398	total: 3.58s	remaining: 3.23s
526:	learn: 0.0797188	total: 3.59s	remaining: 3.22s
527:	learn: 0.0796957	total: 3.6s	remaining: 3.22s
528:	learn: 0.0796957	total: 3.6s	remaining: 3.21s
529:	learn: 0.0796953	total: 3.61s	remaining: 3.2s
530:	learn: 0.0796549	total: 3.61s	remaining: 3.19s
531:	learn: 0.0796127	total: 3.62s	remaining: 3.18s
532:	learn: 0.0794914	total: 3.62s	remaining: 3.17s
533:	learn: 0.0794164	total: 3.63s	remaining: 3.17s
534:	learn: 0.0794145	total: 3.63s	remaining: 3.16s
535:	learn: 0.0794008	total: 3.64s	remaining: 3.15s
536:	learn: 0.0793911	total: 3.65s	remaining: 3.14s
537:	learn: 0.0793859	total: 3.65s	remaining: 3.14s
538:	learn: 0.0792851	total: 3.66s	remaining: 3.13s
539:	learn: 0.0792188	total: 3.67s	remaining: 3.12s
540:	learn: 0.0792158	total: 3.67s	remaining: 3.11s
541:	learn: 0.0

...

192:	learn: 0.1069110	total: 1.2s	remaining: 5.01s
193:	learn: 0.1065335	total: 1.21s	remaining: 5.01s
194:	learn: 0.1063550	total: 1.21s	remaining: 5s
195:	learn: 0.1060765	total: 1.22s	remaining: 5s
196:	learn: 0.1060760	total: 1.22s	remaining: 4.98s
197:	learn: 0.1060752	total: 1.23s	remaining: 4.96s
198:	learn: 0.1060639	total: 1.23s	remaining: 4.96s
199:	learn: 0.1058902	total: 1.24s	remaining: 4.95s
200:	learn: 0.1058311	total: 1.24s	remaining: 4.94s
201:	learn: 0.1056607	total: 1.25s	remaining: 4.94s
202:	learn: 0.1055032	total: 1.26s	remaining: 4.93s
203:	learn: 0.1052486	total: 1.26s	remaining: 4.92s
204:	learn: 0.1052484	total: 1.26s	remaining: 4.91s
205:	learn: 0.1051766	total: 1.27s	remaining: 4.9s
206:	learn: 0.1049892	total: 1.28s	remaining: 4.89s
207:	learn: 0.1048998	total: 1.28s	remaining: 4.89s
208:	learn: 0.1047101	total: 1.29s	remaining: 4.88s
209:	learn: 0.1044432	total: 1.3s	remaining: 4.88s
210:	learn: 0.1042533	total: 1.3s	remaining: 4.87s
211:	learn: 0.1040346	

383:	learn: 0.1001716	total: 1.97s	remaining: 3.16s
384:	learn: 0.1001715	total: 1.97s	remaining: 3.15s
385:	learn: 0.1001715	total: 1.98s	remaining: 3.15s
386:	learn: 0.1001715	total: 1.98s	remaining: 3.14s
387:	learn: 0.1001714	total: 1.99s	remaining: 3.13s
388:	learn: 0.1001711	total: 1.99s	remaining: 3.12s
389:	learn: 0.1001710	total: 1.99s	remaining: 3.12s
390:	learn: 0.1001710	total: 2s	remaining: 3.11s
391:	learn: 0.1001709	total: 2s	remaining: 3.1s
392:	learn: 0.1001709	total: 2s	remaining: 3.09s
393:	learn: 0.1001707	total: 2s	remaining: 3.08s
394:	learn: 0.1001706	total: 2.01s	remaining: 3.07s
395:	learn: 0.1001704	total: 2.01s	remaining: 3.06s
396:	learn: 0.1001704	total: 2.01s	remaining: 3.06s
397:	learn: 0.1001704	total: 2.02s	remaining: 3.05s
398:	learn: 0.1001704	total: 2.02s	remaining: 3.04s
399:	learn: 0.1001703	total: 2.02s	remaining: 3.03s
400:	learn: 0.1001703	total: 2.02s	remaining: 3.02s
401:	learn: 0.1001703	total: 2.03s	remaining: 3.02s
402:	learn: 0.1001702	tot

566:	learn: 0.0933694	total: 2.75s	remaining: 2.1s
567:	learn: 0.0933694	total: 2.75s	remaining: 2.09s
568:	learn: 0.0933694	total: 2.76s	remaining: 2.09s
569:	learn: 0.0933694	total: 2.76s	remaining: 2.08s
570:	learn: 0.0933694	total: 2.76s	remaining: 2.08s
571:	learn: 0.0933694	total: 2.77s	remaining: 2.07s
572:	learn: 0.0933694	total: 2.77s	remaining: 2.06s
573:	learn: 0.0933694	total: 2.77s	remaining: 2.06s
574:	learn: 0.0933694	total: 2.77s	remaining: 2.05s
575:	learn: 0.0933694	total: 2.78s	remaining: 2.04s
576:	learn: 0.0933694	total: 2.78s	remaining: 2.04s
577:	learn: 0.0933693	total: 2.78s	remaining: 2.03s
578:	learn: 0.0933693	total: 2.79s	remaining: 2.03s
579:	learn: 0.0933693	total: 2.79s	remaining: 2.02s
580:	learn: 0.0933693	total: 2.79s	remaining: 2.01s
581:	learn: 0.0933693	total: 2.8s	remaining: 2.01s
582:	learn: 0.0933693	total: 2.8s	remaining: 2s
583:	learn: 0.0933693	total: 2.8s	remaining: 2s
584:	learn: 0.0933693	total: 2.81s	remaining: 1.99s
585:	learn: 0.0933693	

746:	learn: 0.0875016	total: 3.52s	remaining: 1.19s
747:	learn: 0.0875016	total: 3.53s	remaining: 1.19s
748:	learn: 0.0875014	total: 3.53s	remaining: 1.18s
749:	learn: 0.0875012	total: 3.54s	remaining: 1.18s
750:	learn: 0.0875012	total: 3.54s	remaining: 1.17s
751:	learn: 0.0875011	total: 3.54s	remaining: 1.17s
752:	learn: 0.0875010	total: 3.54s	remaining: 1.16s
753:	learn: 0.0875008	total: 3.55s	remaining: 1.16s
754:	learn: 0.0875008	total: 3.55s	remaining: 1.15s
755:	learn: 0.0875008	total: 3.55s	remaining: 1.15s
756:	learn: 0.0875008	total: 3.56s	remaining: 1.14s
757:	learn: 0.0875008	total: 3.56s	remaining: 1.14s
758:	learn: 0.0875004	total: 3.56s	remaining: 1.13s
759:	learn: 0.0875002	total: 3.57s	remaining: 1.13s
760:	learn: 0.0875002	total: 3.57s	remaining: 1.12s
761:	learn: 0.0875001	total: 3.57s	remaining: 1.12s
762:	learn: 0.0875000	total: 3.58s	remaining: 1.11s
763:	learn: 0.0874999	total: 3.58s	remaining: 1.1s
764:	learn: 0.0874998	total: 3.58s	remaining: 1.1s
765:	learn: 0.

911:	learn: 0.0869999	total: 4.09s	remaining: 395ms
912:	learn: 0.0869998	total: 4.1s	remaining: 391ms
913:	learn: 0.0869996	total: 4.1s	remaining: 386ms
914:	learn: 0.0869995	total: 4.11s	remaining: 382ms
915:	learn: 0.0869995	total: 4.11s	remaining: 377ms
916:	learn: 0.0869733	total: 4.12s	remaining: 373ms
917:	learn: 0.0869247	total: 4.13s	remaining: 368ms
918:	learn: 0.0867725	total: 4.13s	remaining: 364ms
919:	learn: 0.0866178	total: 4.14s	remaining: 360ms
920:	learn: 0.0865844	total: 4.14s	remaining: 355ms
921:	learn: 0.0864924	total: 4.15s	remaining: 351ms
922:	learn: 0.0864420	total: 4.16s	remaining: 347ms
923:	learn: 0.0864208	total: 4.16s	remaining: 342ms
924:	learn: 0.0864208	total: 4.17s	remaining: 338ms
925:	learn: 0.0864207	total: 4.17s	remaining: 333ms
926:	learn: 0.0864207	total: 4.17s	remaining: 329ms
927:	learn: 0.0864207	total: 4.18s	remaining: 324ms
928:	learn: 0.0864207	total: 4.18s	remaining: 320ms
929:	learn: 0.0864207	total: 4.18s	remaining: 315ms
930:	learn: 0.

In [51]:
print("Log loss: {:.3f}".format(-log_.mean()))

Log loss: 0.437


In [55]:
opt_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostClassifier(**best_params,od_type='Iter', eval_metric='Logloss'))])

opt_clf.fit(X_train.values, y_train.values.reshape(-1,1))
y_pred = opt_clf.predict_proba(X_test.values)[:, 1]
print("model logloss: %.3f" % log_loss(y_test, y_pred))

/Users/williamlee/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0:	learn: 0.6697220	total: 12.1ms	remaining: 9.16s
1:	learn: 0.6461048	total: 21ms	remaining: 7.96s
2:	learn: 0.6253606	total: 30.3ms	remaining: 7.63s
3:	learn: 0.6026194	total: 39.4ms	remaining: 7.44s
4:	learn: 0.5823713	total: 47.6ms	remaining: 7.18s
5:	learn: 0.5646853	total: 56.3ms	remaining: 7.06s
6:	learn: 0.5427890	total: 64.4ms	remaining: 6.92s
7:	learn: 0.5263117	total: 74.8ms	remaining: 7.02s
8:	learn: 0.5050964	total: 83.6ms	remaining: 6.97s
9:	learn: 0.4910259	total: 92.7ms	remaining: 6.94s
10:	learn: 0.4739767	total: 101ms	remaining: 6.9s
11:	learn: 0.4613058	total: 111ms	remaining: 6.93s
12:	learn: 0.4496900	total: 119ms	remaining: 6.82s
13:	learn: 0.4379222	total: 127ms	remaining: 6.78s
14:	learn: 0.4200535	total: 137ms	remaining: 6.79s
15:	learn: 0.4075263	total: 147ms	remaining: 6.82s
16:	learn: 0.3957930	total: 156ms	remaining: 6.82s
17:	learn: 0.3844064	total: 165ms	remaining: 6.81s
18:	learn: 0.3731632	total: 175ms	remaining: 6.83s
19:	learn: 0.3631718	total: 184ms	

161:	learn: 0.1276554	total: 1.64s	remaining: 6.03s
162:	learn: 0.1275573	total: 1.65s	remaining: 6.02s
163:	learn: 0.1274133	total: 1.66s	remaining: 6.02s
164:	learn: 0.1273094	total: 1.67s	remaining: 6.01s
165:	learn: 0.1272235	total: 1.68s	remaining: 5.99s
166:	learn: 0.1271426	total: 1.69s	remaining: 5.98s
167:	learn: 0.1269909	total: 1.7s	remaining: 5.97s
168:	learn: 0.1268280	total: 1.7s	remaining: 5.95s
169:	learn: 0.1266857	total: 1.71s	remaining: 5.94s
170:	learn: 0.1265945	total: 1.73s	remaining: 5.93s
171:	learn: 0.1264581	total: 1.73s	remaining: 5.92s
172:	learn: 0.1263351	total: 1.75s	remaining: 5.91s
173:	learn: 0.1262526	total: 1.76s	remaining: 5.91s
174:	learn: 0.1261801	total: 1.76s	remaining: 5.89s
175:	learn: 0.1260545	total: 1.77s	remaining: 5.88s
176:	learn: 0.1259311	total: 1.79s	remaining: 5.88s
177:	learn: 0.1258206	total: 1.8s	remaining: 5.87s
178:	learn: 0.1257029	total: 1.81s	remaining: 5.86s
179:	learn: 0.1256298	total: 1.82s	remaining: 5.85s
180:	learn: 0.1

340:	learn: 0.1175038	total: 3.47s	remaining: 4.26s
341:	learn: 0.1174815	total: 3.48s	remaining: 4.25s
342:	learn: 0.1174488	total: 3.49s	remaining: 4.24s
343:	learn: 0.1174256	total: 3.5s	remaining: 4.23s
344:	learn: 0.1173906	total: 3.51s	remaining: 4.21s
345:	learn: 0.1173666	total: 3.52s	remaining: 4.2s
346:	learn: 0.1173308	total: 3.53s	remaining: 4.19s
347:	learn: 0.1172887	total: 3.54s	remaining: 4.18s
348:	learn: 0.1172738	total: 3.55s	remaining: 4.17s
349:	learn: 0.1172455	total: 3.56s	remaining: 4.16s
350:	learn: 0.1172042	total: 3.57s	remaining: 4.15s
351:	learn: 0.1171765	total: 3.58s	remaining: 4.13s
352:	learn: 0.1171601	total: 3.59s	remaining: 4.13s
353:	learn: 0.1171297	total: 3.61s	remaining: 4.13s
354:	learn: 0.1171151	total: 3.62s	remaining: 4.12s
355:	learn: 0.1170924	total: 3.64s	remaining: 4.12s
356:	learn: 0.1170646	total: 3.65s	remaining: 4.11s
357:	learn: 0.1170378	total: 3.66s	remaining: 4.1s
358:	learn: 0.1170192	total: 3.67s	remaining: 4.09s
359:	learn: 0.1

504:	learn: 0.1141477	total: 5.31s	remaining: 2.67s
505:	learn: 0.1141243	total: 5.32s	remaining: 2.66s
506:	learn: 0.1141019	total: 5.33s	remaining: 2.65s
507:	learn: 0.1140764	total: 5.34s	remaining: 2.64s
508:	learn: 0.1140572	total: 5.34s	remaining: 2.63s
509:	learn: 0.1140424	total: 5.35s	remaining: 2.61s
510:	learn: 0.1140280	total: 5.36s	remaining: 2.6s
511:	learn: 0.1140006	total: 5.37s	remaining: 2.59s
512:	learn: 0.1139734	total: 5.38s	remaining: 2.58s
513:	learn: 0.1139557	total: 5.4s	remaining: 2.57s
514:	learn: 0.1139331	total: 5.42s	remaining: 2.57s
515:	learn: 0.1139063	total: 5.43s	remaining: 2.56s
516:	learn: 0.1138838	total: 5.45s	remaining: 2.55s
517:	learn: 0.1138556	total: 5.46s	remaining: 2.54s
518:	learn: 0.1138414	total: 5.48s	remaining: 2.53s
519:	learn: 0.1138099	total: 5.49s	remaining: 2.52s
520:	learn: 0.1137878	total: 5.51s	remaining: 2.52s
521:	learn: 0.1137657	total: 5.52s	remaining: 2.51s
522:	learn: 0.1137566	total: 5.54s	remaining: 2.5s
523:	learn: 0.1

681:	learn: 0.1106971	total: 7.12s	remaining: 804ms
682:	learn: 0.1106705	total: 7.13s	remaining: 794ms
683:	learn: 0.1106401	total: 7.15s	remaining: 784ms
684:	learn: 0.1106275	total: 7.16s	remaining: 774ms
685:	learn: 0.1106119	total: 7.18s	remaining: 764ms
686:	learn: 0.1106032	total: 7.19s	remaining: 753ms
687:	learn: 0.1105781	total: 7.2s	remaining: 743ms
688:	learn: 0.1105638	total: 7.21s	remaining: 732ms
689:	learn: 0.1105493	total: 7.21s	remaining: 721ms
690:	learn: 0.1105369	total: 7.22s	remaining: 711ms
691:	learn: 0.1105068	total: 7.23s	remaining: 700ms
692:	learn: 0.1104911	total: 7.24s	remaining: 690ms
693:	learn: 0.1104771	total: 7.25s	remaining: 679ms
694:	learn: 0.1104650	total: 7.26s	remaining: 669ms
695:	learn: 0.1104485	total: 7.27s	remaining: 658ms
696:	learn: 0.1104188	total: 7.28s	remaining: 648ms
697:	learn: 0.1104145	total: 7.29s	remaining: 637ms
698:	learn: 0.1103983	total: 7.3s	remaining: 626ms
699:	learn: 0.1103805	total: 7.31s	remaining: 616ms
700:	learn: 0.

In [23]:
opt_cat = CatBoostClassifier(**best_params,od_type='Iter', loss_function='Logloss', cat_features=categorical_f)
# opt_cat.fit(X_train_re, y_train_re, cat_features=categorical_f)

log_2 = cross_val_score(estimator = opt_cat, X = X_train_re, y = y_train_re, 
                             cv = 5, scoring=LogLoss)

0:	learn: 0.6865870	total: 41ms	remaining: 40.9s
1:	learn: 0.6803749	total: 79.4ms	remaining: 39.6s
2:	learn: 0.6742512	total: 119ms	remaining: 39.5s
3:	learn: 0.6683201	total: 156ms	remaining: 38.9s
4:	learn: 0.6624911	total: 190ms	remaining: 37.8s
5:	learn: 0.6568084	total: 231ms	remaining: 38.4s
6:	learn: 0.6514418	total: 255ms	remaining: 36.2s
7:	learn: 0.6460545	total: 278ms	remaining: 34.5s
8:	learn: 0.6408656	total: 316ms	remaining: 34.8s
9:	learn: 0.6357314	total: 370ms	remaining: 36.6s
10:	learn: 0.6306905	total: 433ms	remaining: 39s
11:	learn: 0.6256938	total: 461ms	remaining: 38s
12:	learn: 0.6209260	total: 500ms	remaining: 38s
13:	learn: 0.6163016	total: 541ms	remaining: 38.1s
14:	learn: 0.6118400	total: 578ms	remaining: 37.9s
15:	learn: 0.6074414	total: 614ms	remaining: 37.8s
16:	learn: 0.6031800	total: 652ms	remaining: 37.7s
17:	learn: 0.5990325	total: 693ms	remaining: 37.8s
18:	learn: 0.5948505	total: 734ms	remaining: 37.9s
19:	learn: 0.5843899	total: 778ms	remaining: 38

164:	learn: 0.1762573	total: 8s	remaining: 40.5s
165:	learn: 0.1756429	total: 8.05s	remaining: 40.5s
166:	learn: 0.1750420	total: 8.1s	remaining: 40.4s
167:	learn: 0.1744571	total: 8.15s	remaining: 40.4s
168:	learn: 0.1739060	total: 8.2s	remaining: 40.3s
169:	learn: 0.1733683	total: 8.25s	remaining: 40.3s
170:	learn: 0.1728081	total: 8.3s	remaining: 40.3s
171:	learn: 0.1722671	total: 8.36s	remaining: 40.2s
172:	learn: 0.1717442	total: 8.4s	remaining: 40.2s
173:	learn: 0.1712161	total: 8.45s	remaining: 40.1s
174:	learn: 0.1707147	total: 8.5s	remaining: 40.1s
175:	learn: 0.1701831	total: 8.56s	remaining: 40.1s
176:	learn: 0.1696047	total: 8.61s	remaining: 40s
177:	learn: 0.1691396	total: 8.67s	remaining: 40s
178:	learn: 0.1685858	total: 8.72s	remaining: 40s
179:	learn: 0.1681419	total: 8.78s	remaining: 40s
180:	learn: 0.1676683	total: 8.84s	remaining: 40s
181:	learn: 0.1671478	total: 8.89s	remaining: 40s
182:	learn: 0.1666591	total: 8.95s	remaining: 39.9s
183:	learn: 0.1662493	total: 8.9

323:	learn: 0.1428015	total: 15.7s	remaining: 32.8s
324:	learn: 0.1427936	total: 15.8s	remaining: 32.7s
325:	learn: 0.1427905	total: 15.8s	remaining: 32.7s
326:	learn: 0.1426750	total: 15.8s	remaining: 32.6s
327:	learn: 0.1426411	total: 15.9s	remaining: 32.5s
328:	learn: 0.1425512	total: 15.9s	remaining: 32.5s
329:	learn: 0.1425420	total: 16s	remaining: 32.4s
330:	learn: 0.1424841	total: 16s	remaining: 32.4s
331:	learn: 0.1424803	total: 16.1s	remaining: 32.3s
332:	learn: 0.1424710	total: 16.1s	remaining: 32.3s
333:	learn: 0.1424632	total: 16.2s	remaining: 32.2s
334:	learn: 0.1424530	total: 16.2s	remaining: 32.2s
335:	learn: 0.1424395	total: 16.2s	remaining: 32.1s
336:	learn: 0.1423285	total: 16.3s	remaining: 32s
337:	learn: 0.1423239	total: 16.3s	remaining: 32s
338:	learn: 0.1423152	total: 16.4s	remaining: 31.9s
339:	learn: 0.1423096	total: 16.4s	remaining: 31.8s
340:	learn: 0.1423010	total: 16.4s	remaining: 31.8s
341:	learn: 0.1422925	total: 16.5s	remaining: 31.7s
342:	learn: 0.142283

482:	learn: 0.1392404	total: 22.4s	remaining: 24s
483:	learn: 0.1392377	total: 22.5s	remaining: 24s
484:	learn: 0.1392259	total: 22.5s	remaining: 23.9s
485:	learn: 0.1392233	total: 22.6s	remaining: 23.9s
486:	learn: 0.1392208	total: 22.6s	remaining: 23.8s
487:	learn: 0.1392042	total: 22.6s	remaining: 23.8s
488:	learn: 0.1391829	total: 22.7s	remaining: 23.7s
489:	learn: 0.1391802	total: 22.7s	remaining: 23.7s
490:	learn: 0.1391771	total: 22.8s	remaining: 23.6s
491:	learn: 0.1391562	total: 22.8s	remaining: 23.6s
492:	learn: 0.1391541	total: 22.9s	remaining: 23.5s
493:	learn: 0.1391490	total: 22.9s	remaining: 23.5s
494:	learn: 0.1391380	total: 22.9s	remaining: 23.4s
495:	learn: 0.1391249	total: 23s	remaining: 23.4s
496:	learn: 0.1391218	total: 23s	remaining: 23.3s
497:	learn: 0.1391208	total: 23.1s	remaining: 23.3s
498:	learn: 0.1391182	total: 23.1s	remaining: 23.2s
499:	learn: 0.1391156	total: 23.1s	remaining: 23.1s
500:	learn: 0.1391124	total: 23.2s	remaining: 23.1s
501:	learn: 0.139109

642:	learn: 0.1357009	total: 29.7s	remaining: 16.5s
643:	learn: 0.1356629	total: 29.8s	remaining: 16.5s
644:	learn: 0.1356487	total: 29.8s	remaining: 16.4s
645:	learn: 0.1356071	total: 29.9s	remaining: 16.4s
646:	learn: 0.1355975	total: 29.9s	remaining: 16.3s
647:	learn: 0.1355902	total: 30s	remaining: 16.3s
648:	learn: 0.1355721	total: 30s	remaining: 16.2s
649:	learn: 0.1355465	total: 30.1s	remaining: 16.2s
650:	learn: 0.1355230	total: 30.1s	remaining: 16.1s
651:	learn: 0.1355143	total: 30.2s	remaining: 16.1s
652:	learn: 0.1354926	total: 30.2s	remaining: 16s
653:	learn: 0.1354661	total: 30.2s	remaining: 16s
654:	learn: 0.1354260	total: 30.3s	remaining: 16s
655:	learn: 0.1354073	total: 30.3s	remaining: 15.9s
656:	learn: 0.1354033	total: 30.4s	remaining: 15.9s
657:	learn: 0.1353662	total: 30.4s	remaining: 15.8s
658:	learn: 0.1353599	total: 30.5s	remaining: 15.8s
659:	learn: 0.1353351	total: 30.5s	remaining: 15.7s
660:	learn: 0.1353000	total: 30.6s	remaining: 15.7s
661:	learn: 0.1352886	

...

352:	learn: 0.1338843	total: 18.3s	remaining: 33.5s
353:	learn: 0.1338353	total: 18.3s	remaining: 33.5s
354:	learn: 0.1337854	total: 18.4s	remaining: 33.4s
355:	learn: 0.1337400	total: 18.4s	remaining: 33.4s
356:	learn: 0.1336773	total: 18.5s	remaining: 33.3s
357:	learn: 0.1336449	total: 18.6s	remaining: 33.3s
358:	learn: 0.1336303	total: 18.6s	remaining: 33.2s
359:	learn: 0.1335857	total: 18.7s	remaining: 33.2s
360:	learn: 0.1335732	total: 18.7s	remaining: 33.1s
361:	learn: 0.1335591	total: 18.8s	remaining: 33.1s
362:	learn: 0.1334814	total: 18.8s	remaining: 33s
363:	learn: 0.1334224	total: 18.9s	remaining: 33s
364:	learn: 0.1334006	total: 18.9s	remaining: 32.9s
365:	learn: 0.1333859	total: 19s	remaining: 32.9s
366:	learn: 0.1333438	total: 19s	remaining: 32.8s
367:	learn: 0.1333265	total: 19.1s	remaining: 32.7s
368:	learn: 0.1333071	total: 19.1s	remaining: 32.7s
369:	learn: 0.1332653	total: 19.2s	remaining: 32.6s
370:	learn: 0.1332505	total: 19.2s	remaining: 32.6s
371:	learn: 0.133245

512:	learn: 0.1298283	total: 26.1s	remaining: 24.8s
513:	learn: 0.1298066	total: 26.1s	remaining: 24.7s
514:	learn: 0.1297847	total: 26.2s	remaining: 24.6s
515:	learn: 0.1297602	total: 26.2s	remaining: 24.6s
516:	learn: 0.1297388	total: 26.3s	remaining: 24.5s
517:	learn: 0.1297353	total: 26.3s	remaining: 24.5s
518:	learn: 0.1297285	total: 26.3s	remaining: 24.4s
519:	learn: 0.1297240	total: 26.4s	remaining: 24.4s
520:	learn: 0.1296521	total: 26.4s	remaining: 24.3s
521:	learn: 0.1296359	total: 26.5s	remaining: 24.3s
522:	learn: 0.1295836	total: 26.6s	remaining: 24.2s
523:	learn: 0.1295784	total: 26.6s	remaining: 24.2s
524:	learn: 0.1295582	total: 26.6s	remaining: 24.1s
525:	learn: 0.1295553	total: 26.7s	remaining: 24.1s
526:	learn: 0.1295473	total: 26.7s	remaining: 24s
527:	learn: 0.1295395	total: 26.8s	remaining: 23.9s
528:	learn: 0.1295214	total: 26.8s	remaining: 23.9s
529:	learn: 0.1295036	total: 26.9s	remaining: 23.8s
530:	learn: 0.1294977	total: 26.9s	remaining: 23.8s
531:	learn: 0.

671:	learn: 0.1266665	total: 33.6s	remaining: 16.4s
672:	learn: 0.1266623	total: 33.6s	remaining: 16.3s
673:	learn: 0.1266260	total: 33.7s	remaining: 16.3s
674:	learn: 0.1266026	total: 33.7s	remaining: 16.2s
675:	learn: 0.1265978	total: 33.8s	remaining: 16.2s
676:	learn: 0.1265687	total: 33.8s	remaining: 16.1s
677:	learn: 0.1265492	total: 33.9s	remaining: 16.1s
678:	learn: 0.1265432	total: 33.9s	remaining: 16s
679:	learn: 0.1265392	total: 34s	remaining: 16s
680:	learn: 0.1265353	total: 34s	remaining: 15.9s
681:	learn: 0.1264996	total: 34.1s	remaining: 15.9s
682:	learn: 0.1264686	total: 34.1s	remaining: 15.8s
683:	learn: 0.1264479	total: 34.2s	remaining: 15.8s
684:	learn: 0.1264283	total: 34.2s	remaining: 15.7s
685:	learn: 0.1264224	total: 34.3s	remaining: 15.7s
686:	learn: 0.1264062	total: 34.3s	remaining: 15.6s
687:	learn: 0.1263918	total: 34.3s	remaining: 15.6s
688:	learn: 0.1263856	total: 34.4s	remaining: 15.5s
689:	learn: 0.1263819	total: 34.4s	remaining: 15.5s
690:	learn: 0.126351

831:	learn: 0.1233389	total: 41.5s	remaining: 8.37s
832:	learn: 0.1233312	total: 41.5s	remaining: 8.32s
833:	learn: 0.1233267	total: 41.6s	remaining: 8.27s
834:	learn: 0.1233019	total: 41.6s	remaining: 8.22s
835:	learn: 0.1232607	total: 41.7s	remaining: 8.18s
836:	learn: 0.1232551	total: 41.7s	remaining: 8.13s
837:	learn: 0.1232184	total: 41.8s	remaining: 8.08s
838:	learn: 0.1232141	total: 41.8s	remaining: 8.03s
839:	learn: 0.1232108	total: 41.9s	remaining: 7.97s
840:	learn: 0.1231577	total: 41.9s	remaining: 7.93s
841:	learn: 0.1231450	total: 42s	remaining: 7.88s
842:	learn: 0.1231412	total: 42s	remaining: 7.83s
843:	learn: 0.1231118	total: 42.1s	remaining: 7.77s
844:	learn: 0.1230682	total: 42.1s	remaining: 7.72s
845:	learn: 0.1230361	total: 42.2s	remaining: 7.68s
846:	learn: 0.1229931	total: 42.2s	remaining: 7.63s
847:	learn: 0.1229916	total: 42.3s	remaining: 7.58s
848:	learn: 0.1229563	total: 42.3s	remaining: 7.53s
849:	learn: 0.1229517	total: 42.4s	remaining: 7.48s
850:	learn: 0.12

990:	learn: 0.1202613	total: 49.5s	remaining: 449ms
991:	learn: 0.1202158	total: 49.5s	remaining: 399ms
992:	learn: 0.1201917	total: 49.6s	remaining: 350ms
993:	learn: 0.1201823	total: 49.6s	remaining: 300ms
994:	learn: 0.1201734	total: 49.7s	remaining: 250ms
995:	learn: 0.1201687	total: 49.8s	remaining: 200ms
996:	learn: 0.1201664	total: 49.8s	remaining: 150ms
997:	learn: 0.1201365	total: 49.8s	remaining: 99.9ms
998:	learn: 0.1201269	total: 49.9s	remaining: 49.9ms
999:	learn: 0.1201181	total: 49.9s	remaining: 0us


In [24]:
print("Log loss: {:.3f}".format(log_2.mean()))

Log loss: -0.189


In [69]:
opt_cat = CatBoostClassifier(**best_params,od_type='Iter', loss_function='Logloss', cat_features=categorical_f)
opt_cat.fit(X_train, y_train)
y_pred_cat = opt_cat.predict_proba(X_test.values)[:, 1]
print("model logloss: %.3f" % log_loss(y_test, y_pred_cat))

0:	learn: 0.6872590	total: 2.98s	remaining: 37m 35s
1:	learn: 0.6815207	total: 5.03s	remaining: 31m 44s
2:	learn: 0.6759145	total: 6.83s	remaining: 28m 42s
3:	learn: 0.6705470	total: 8.7s	remaining: 27m 21s
4:	learn: 0.6652111	total: 10.5s	remaining: 26m 27s
5:	learn: 0.6601074	total: 12.4s	remaining: 25m 50s
6:	learn: 0.6551261	total: 14.2s	remaining: 25m 25s
7:	learn: 0.6502594	total: 16.1s	remaining: 25m 10s
8:	learn: 0.6455273	total: 18s	remaining: 25m 1s
9:	learn: 0.6409089	total: 20.2s	remaining: 25m 13s
10:	learn: 0.6364029	total: 22.1s	remaining: 25m 2s
11:	learn: 0.6319297	total: 23.7s	remaining: 24m 34s
12:	learn: 0.6276322	total: 25.8s	remaining: 24m 37s
13:	learn: 0.6233885	total: 27.5s	remaining: 24m 23s
14:	learn: 0.6192511	total: 29.1s	remaining: 24m 2s
15:	learn: 0.6152651	total: 31.1s	remaining: 24m 5s
16:	learn: 0.6113927	total: 33.2s	remaining: 24m 7s
17:	learn: 0.6076081	total: 35.2s	remaining: 24m 10s
18:	learn: 0.6039616	total: 37.1s	remaining: 24m 5s
19:	learn: 0

153:	learn: 0.4542701	total: 5m 42s	remaining: 22m 24s
154:	learn: 0.4540435	total: 5m 44s	remaining: 22m 22s
155:	learn: 0.4538059	total: 5m 46s	remaining: 22m 18s
156:	learn: 0.4535863	total: 5m 47s	remaining: 22m 10s
157:	learn: 0.4533752	total: 5m 48s	remaining: 22m 7s
158:	learn: 0.4531693	total: 5m 50s	remaining: 22m 3s
159:	learn: 0.4527284	total: 5m 53s	remaining: 22m 5s
160:	learn: 0.4522976	total: 5m 56s	remaining: 22m 4s
161:	learn: 0.4518767	total: 5m 59s	remaining: 22m 4s
162:	learn: 0.4514611	total: 6m 2s	remaining: 22m 4s
163:	learn: 0.4510466	total: 6m 6s	remaining: 22m 11s
164:	learn: 0.4506314	total: 6m 11s	remaining: 22m 16s
165:	learn: 0.4502258	total: 6m 15s	remaining: 22m 20s
166:	learn: 0.4498586	total: 6m 19s	remaining: 22m 24s
167:	learn: 0.4494706	total: 6m 23s	remaining: 22m 28s
168:	learn: 0.4490998	total: 6m 27s	remaining: 22m 33s
169:	learn: 0.4487373	total: 6m 31s	remaining: 22m 37s
170:	learn: 0.4483832	total: 6m 35s	remaining: 22m 41s
171:	learn: 0.4480

303:	learn: 0.4313955	total: 16m 17s	remaining: 24m 23s
304:	learn: 0.4313479	total: 16m 22s	remaining: 24m 21s
305:	learn: 0.4313150	total: 16m 26s	remaining: 24m 20s
306:	learn: 0.4312795	total: 16m 30s	remaining: 24m 18s
307:	learn: 0.4312312	total: 16m 34s	remaining: 24m 16s
308:	learn: 0.4311954	total: 16m 39s	remaining: 24m 16s
309:	learn: 0.4311518	total: 16m 44s	remaining: 24m 14s
310:	learn: 0.4311191	total: 16m 48s	remaining: 24m 12s
311:	learn: 0.4310785	total: 16m 53s	remaining: 24m 11s
312:	learn: 0.4310495	total: 16m 58s	remaining: 24m 10s
313:	learn: 0.4310096	total: 17m 3s	remaining: 24m 10s
314:	learn: 0.4309788	total: 17m 7s	remaining: 24m 8s
315:	learn: 0.4309433	total: 17m 12s	remaining: 24m 7s
316:	learn: 0.4309043	total: 17m 16s	remaining: 24m 5s
317:	learn: 0.4308675	total: 17m 22s	remaining: 24m 5s
318:	learn: 0.4308354	total: 17m 26s	remaining: 24m 3s
319:	learn: 0.4307969	total: 17m 31s	remaining: 24m 2s
320:	learn: 0.4307621	total: 17m 36s	remaining: 24m 1s
3

451:	learn: 0.4280462	total: 26m 30s	remaining: 18m
452:	learn: 0.4280334	total: 26m 33s	remaining: 17m 56s
453:	learn: 0.4280158	total: 26m 37s	remaining: 17m 53s
454:	learn: 0.4280030	total: 26m 41s	remaining: 17m 49s
455:	learn: 0.4279908	total: 26m 43s	remaining: 17m 45s
456:	learn: 0.4279809	total: 26m 47s	remaining: 17m 42s
457:	learn: 0.4279712	total: 26m 51s	remaining: 17m 39s
458:	learn: 0.4279617	total: 26m 55s	remaining: 17m 35s
459:	learn: 0.4279521	total: 26m 59s	remaining: 17m 32s
460:	learn: 0.4279364	total: 27m 3s	remaining: 17m 29s
461:	learn: 0.4279247	total: 27m 7s	remaining: 17m 26s
462:	learn: 0.4279083	total: 27m 11s	remaining: 17m 22s
463:	learn: 0.4278993	total: 27m 15s	remaining: 17m 19s
464:	learn: 0.4278901	total: 27m 19s	remaining: 17m 16s
465:	learn: 0.4278788	total: 27m 21s	remaining: 17m 12s
466:	learn: 0.4278699	total: 27m 25s	remaining: 17m 8s
467:	learn: 0.4278569	total: 27m 28s	remaining: 17m 4s
468:	learn: 0.4278459	total: 27m 30s	remaining: 17m
469:

599:	learn: 0.4266669	total: 35m 28s	remaining: 9m 24s
600:	learn: 0.4266605	total: 35m 32s	remaining: 9m 20s
601:	learn: 0.4266543	total: 35m 37s	remaining: 9m 17s
602:	learn: 0.4266481	total: 35m 39s	remaining: 9m 13s
603:	learn: 0.4266421	total: 35m 44s	remaining: 9m 10s
604:	learn: 0.4266365	total: 35m 47s	remaining: 9m 6s
605:	learn: 0.4266278	total: 35m 50s	remaining: 9m 3s
606:	learn: 0.4266203	total: 35m 53s	remaining: 8m 59s
607:	learn: 0.4266157	total: 35m 56s	remaining: 8m 55s
608:	learn: 0.4266080	total: 36m	remaining: 8m 52s
609:	learn: 0.4266035	total: 36m 3s	remaining: 8m 48s
610:	learn: 0.4265966	total: 36m 7s	remaining: 8m 44s
611:	learn: 0.4265895	total: 36m 9s	remaining: 8m 41s
612:	learn: 0.4265849	total: 36m 12s	remaining: 8m 37s
613:	learn: 0.4265800	total: 36m 13s	remaining: 8m 33s
614:	learn: 0.4265752	total: 36m 16s	remaining: 8m 29s
615:	learn: 0.4265669	total: 36m 21s	remaining: 8m 26s
616:	learn: 0.4265622	total: 36m 24s	remaining: 8m 22s
617:	learn: 0.42655

750:	learn: 0.4258830	total: 44m 35s	remaining: 28.5s
751:	learn: 0.4258793	total: 44m 37s	remaining: 24.9s
752:	learn: 0.4258756	total: 44m 41s	remaining: 21.4s
753:	learn: 0.4258651	total: 44m 47s	remaining: 17.8s
754:	learn: 0.4258562	total: 44m 51s	remaining: 14.3s
755:	learn: 0.4258460	total: 44m 56s	remaining: 10.7s
756:	learn: 0.4258362	total: 45m	remaining: 7.13s
757:	learn: 0.4258286	total: 45m 5s	remaining: 3.57s
758:	learn: 0.4258188	total: 45m 10s	remaining: 0us
model logloss: 0.417


#### XGBoost Training

In [59]:
df_train

,click,hour,C1,banner_pos,site_domain,site_category,app_domain,app_category,device_id,device_ip,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,14102100,5,0,dd641cc7,8fd0aea4,2021c8a9,07d7df22,a99f214a,ddd2926e,...,1,2,15706,201,31,1300,0,3,-1,67
1,0,14102100,5,0,dd641cc7,8fd0aea4,2021c8a9,07d7df22,a99f214a,96809ac8,...,1,0,15704,201,31,1300,0,3,85,67
2,0,14102100,5,0,dd641cc7,8fd0aea4,2021c8a9,07d7df22,a99f214a,b3cf8def,...,1,0,15704,201,31,1300,0,3,85,67
3,0,14102100,5,0,dd641cc7,8fd0aea4,2021c8a9,07d7df22,a99f214a,e8275b8f,...,1,0,15706,201,31,1300,0,3,85,67
4,0,14102100,5,1,cbee4b41,72722551,2021c8a9,07d7df22,a99f214a,9644d0bf,...,1,0,18993,201,31,1739,0,3,-1,145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,14102100,5,0,dd641cc7,8fd0aea4,2021c8a9,07d7df22,a99f214a,fd4045ce,...,1,0,15702,201,31,1300,0,3,-1,67
19996,0,14102100,5,0,dd641cc7,8fd0aea4,2021c8a9,07d7df22,a99f214a,f5fe9cc8,...,1,0,15708,201,31,1300,0,3,85,67
19997,0,14102100,5,0,dd641cc7,8fd0aea4,2021c8a9,07d7df22,a99f214a,6bcbcc53,...,1,2,15708,201,31,1300,0,3,-1,67
19998,0,14102100,5,0,78590e4e,74073276,2021c8a9,07d7df22,a99f214a,e5025233,...,1,0,15699,201,31,1300,0,3,-1,67


### lightgbm

In [120]:
df_lgb = df_copy.copy()
def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self
df_lgb = convert_obj_to_int(df_lgb)

X_train_lgb, X_test_lgb, y_train_lgb, y_test_lgb = train_test_split(df_lgb, Y, test_size=0.3)

In [125]:
import lightgbm as lgb
msk = np.random.rand(len(X_train)) < 0.8
lgb_train = lgb.Dataset(X_train_lgb[msk], y_train_lgb[msk])
lgb_eval = lgb.Dataset(X_train_lgb[~msk], y_train_lgb[~msk], reference=lgb_train)

In [126]:
params_lgb = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'max_depth': 6,
    'learning_rate': 0.06,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_bin': 255,
    'verbose': 30
}

In [128]:
categorical_f = ['C1','device_type','device_conn_type','C18','banner_pos','site_domain','site_category',
                 'device_id','device_ip','device_model','C14','C15','C16','C17','C19','C20','C21']

lgb_model = lgb.LGBMClassifier()
gbm = lgb.train(params_lgb,
                lgb_train,
                num_boost_round=120)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 2117, number of negative: 9003
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.911331
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.428862
[LightGBM] [Debug] init for col-wise cost 0.001513 seconds, init for row-wise cost 0.006227 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002117 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM

### Bayesian Search CV for lightgbm

In [134]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer, auc, log_loss, roc_auc_score

import lightgbm as lgb

# categorical_f = [ca for ca in X_train.columns if X_train[ca].dtype == 'object']

param = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'max_depth': Integer(3, 7),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_bin': Integer(20, 255, 'uniform'),
    'n_estimators': Integer(100, 1000, 'uniform'),
    'num_leaves': Integer(24, 80, 'uniform')
}

# lgb_model = lgb.LGBMClassifier()

# log-uniform: understand as search over p = exp(x) by varying x
opt_xgb = BayesSearchCV(
    lgb,
    param,
    scoring = LogLoss,
    n_iter=32,
    cv=5,
    random_state=42
)

# executes bayesian optimization
opt_xgb.fit(lgb_train)

ValueError: Dimension has to be a list or tuple.

In [ ]:

def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=3, random_seed=6,n_estimators=10000, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, min_data_in_leaf,min_sum_hessian_in_leaf,subsample):
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)

In [ ]:
def random_search(param_grid, out_file, max_evals = MAX_EVALS):
    """Random search for hyperparameter optimization. 
       Writes result of search to csv file every search iteration."""
    
    
    # Dataframe for results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                                  index = list(range(MAX_EVALS)))
    for i in range(MAX_EVALS):
        
        # Choose random hyperparameters
        random_params = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}
        random_params['subsample'] = 1.0 if random_params['boosting_type'] == 'goss' else random_params['subsample']

        # Evaluate randomly selected hyperparameters
        eval_results = objective(random_params, i)
        results.loc[i, :] = eval_results

        # open connection (append option) and write results
        of_connection = open(out_file, 'a')
        writer = csv.writer(of_connection)
        writer.writerow(eval_results)
        
        # make sure to close connection
        of_connection.close()
        
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)

    return results 

### Output_1

In [33]:
opt_cat = CatBoostClassifier(**best_params,od_type='Iter', loss_function='Logloss', cat_features=categorical_f)
opt_cat.fit(df_copy, Y)

0:	learn: 0.6876016	total: 37.8ms	remaining: 37.8s
1:	learn: 0.6821458	total: 77.9ms	remaining: 38.9s
2:	learn: 0.6768222	total: 114ms	remaining: 38s
3:	learn: 0.6716256	total: 150ms	remaining: 37.4s
4:	learn: 0.6665317	total: 186ms	remaining: 37s
5:	learn: 0.6615421	total: 224ms	remaining: 37.1s
6:	learn: 0.6567268	total: 270ms	remaining: 38.3s
7:	learn: 0.6520342	total: 309ms	remaining: 38.4s
8:	learn: 0.6474125	total: 345ms	remaining: 38s
9:	learn: 0.6428989	total: 381ms	remaining: 37.7s
10:	learn: 0.6384977	total: 413ms	remaining: 37.1s
11:	learn: 0.6341903	total: 446ms	remaining: 36.8s
12:	learn: 0.6299779	total: 482ms	remaining: 36.6s
13:	learn: 0.6258756	total: 523ms	remaining: 36.8s
14:	learn: 0.6218983	total: 560ms	remaining: 36.8s
15:	learn: 0.6179753	total: 599ms	remaining: 36.8s
16:	learn: 0.6142200	total: 633ms	remaining: 36.6s
17:	learn: 0.6104715	total: 666ms	remaining: 36.3s
18:	learn: 0.6068361	total: 707ms	remaining: 36.5s
19:	learn: 0.6032959	total: 754ms	remaining: 

163:	learn: 0.4467688	total: 7.01s	remaining: 35.7s
164:	learn: 0.4465398	total: 7.06s	remaining: 35.7s
165:	learn: 0.4463409	total: 7.11s	remaining: 35.7s
166:	learn: 0.4461307	total: 7.16s	remaining: 35.7s
167:	learn: 0.4459211	total: 7.2s	remaining: 35.7s
168:	learn: 0.4457363	total: 7.25s	remaining: 35.7s
169:	learn: 0.4455335	total: 7.31s	remaining: 35.7s
170:	learn: 0.4453196	total: 7.36s	remaining: 35.7s
171:	learn: 0.4451401	total: 7.41s	remaining: 35.7s
172:	learn: 0.4449336	total: 7.46s	remaining: 35.7s
173:	learn: 0.4447397	total: 7.51s	remaining: 35.6s
174:	learn: 0.4446155	total: 7.55s	remaining: 35.6s
175:	learn: 0.4444641	total: 7.6s	remaining: 35.6s
176:	learn: 0.4443120	total: 7.65s	remaining: 35.6s
177:	learn: 0.4441577	total: 7.72s	remaining: 35.7s
178:	learn: 0.4440145	total: 7.79s	remaining: 35.7s
179:	learn: 0.4438734	total: 7.85s	remaining: 35.8s
180:	learn: 0.4436916	total: 7.9s	remaining: 35.8s
181:	learn: 0.4435609	total: 7.95s	remaining: 35.7s
182:	learn: 0.4

325:	learn: 0.4251704	total: 15.4s	remaining: 31.8s
326:	learn: 0.4250799	total: 15.4s	remaining: 31.8s
327:	learn: 0.4249846	total: 15.5s	remaining: 31.7s
328:	learn: 0.4249004	total: 15.6s	remaining: 31.7s
329:	learn: 0.4248181	total: 15.6s	remaining: 31.7s
330:	learn: 0.4247516	total: 15.7s	remaining: 31.7s
331:	learn: 0.4246609	total: 15.8s	remaining: 31.7s
332:	learn: 0.4245710	total: 15.8s	remaining: 31.7s
333:	learn: 0.4245002	total: 15.9s	remaining: 31.7s
334:	learn: 0.4243986	total: 15.9s	remaining: 31.6s
335:	learn: 0.4242974	total: 16s	remaining: 31.6s
336:	learn: 0.4242170	total: 16.1s	remaining: 31.6s
337:	learn: 0.4241757	total: 16.1s	remaining: 31.6s
338:	learn: 0.4240839	total: 16.2s	remaining: 31.6s
339:	learn: 0.4240379	total: 16.2s	remaining: 31.5s
340:	learn: 0.4239635	total: 16.3s	remaining: 31.5s
341:	learn: 0.4239126	total: 16.4s	remaining: 31.5s
342:	learn: 0.4238606	total: 16.4s	remaining: 31.4s
343:	learn: 0.4237977	total: 16.5s	remaining: 31.4s
344:	learn: 0.

484:	learn: 0.4183183	total: 24.6s	remaining: 26.1s
485:	learn: 0.4182884	total: 24.7s	remaining: 26.1s
486:	learn: 0.4182569	total: 24.7s	remaining: 26s
487:	learn: 0.4182362	total: 24.8s	remaining: 26s
488:	learn: 0.4182184	total: 24.8s	remaining: 25.9s
489:	learn: 0.4181925	total: 24.9s	remaining: 25.9s
490:	learn: 0.4181637	total: 24.9s	remaining: 25.8s
491:	learn: 0.4181346	total: 25s	remaining: 25.8s
492:	learn: 0.4181060	total: 25s	remaining: 25.7s
493:	learn: 0.4180996	total: 25.1s	remaining: 25.7s
494:	learn: 0.4180321	total: 25.1s	remaining: 25.6s
495:	learn: 0.4180217	total: 25.2s	remaining: 25.6s
496:	learn: 0.4179978	total: 25.2s	remaining: 25.5s
497:	learn: 0.4179654	total: 25.3s	remaining: 25.5s
498:	learn: 0.4179464	total: 25.3s	remaining: 25.4s
499:	learn: 0.4178968	total: 25.4s	remaining: 25.4s
500:	learn: 0.4178813	total: 25.4s	remaining: 25.3s
501:	learn: 0.4178577	total: 25.5s	remaining: 25.3s
502:	learn: 0.4178389	total: 25.5s	remaining: 25.2s
503:	learn: 0.417817

643:	learn: 0.4143940	total: 33.2s	remaining: 18.3s
644:	learn: 0.4143369	total: 33.2s	remaining: 18.3s
645:	learn: 0.4143307	total: 33.3s	remaining: 18.2s
646:	learn: 0.4142956	total: 33.3s	remaining: 18.2s
647:	learn: 0.4142874	total: 33.4s	remaining: 18.1s
648:	learn: 0.4142300	total: 33.4s	remaining: 18.1s
649:	learn: 0.4142098	total: 33.5s	remaining: 18s
650:	learn: 0.4142018	total: 33.5s	remaining: 18s
651:	learn: 0.4141751	total: 33.6s	remaining: 17.9s
652:	learn: 0.4141712	total: 33.6s	remaining: 17.9s
653:	learn: 0.4141331	total: 33.7s	remaining: 17.8s
654:	learn: 0.4140896	total: 33.7s	remaining: 17.8s
655:	learn: 0.4140503	total: 33.8s	remaining: 17.7s
656:	learn: 0.4140323	total: 33.8s	remaining: 17.7s
657:	learn: 0.4140286	total: 33.9s	remaining: 17.6s
658:	learn: 0.4140165	total: 34s	remaining: 17.6s
659:	learn: 0.4140035	total: 34s	remaining: 17.5s
660:	learn: 0.4139874	total: 34.1s	remaining: 17.5s
661:	learn: 0.4139668	total: 34.1s	remaining: 17.4s
662:	learn: 0.413924

802:	learn: 0.4103689	total: 41.7s	remaining: 10.2s
803:	learn: 0.4103639	total: 41.7s	remaining: 10.2s
804:	learn: 0.4103431	total: 41.8s	remaining: 10.1s
805:	learn: 0.4102792	total: 41.8s	remaining: 10.1s
806:	learn: 0.4102618	total: 41.9s	remaining: 10s
807:	learn: 0.4102476	total: 41.9s	remaining: 9.96s
808:	learn: 0.4101754	total: 42s	remaining: 9.92s
809:	learn: 0.4101689	total: 42.1s	remaining: 9.86s
810:	learn: 0.4101160	total: 42.1s	remaining: 9.81s
811:	learn: 0.4100946	total: 42.2s	remaining: 9.76s
812:	learn: 0.4100718	total: 42.2s	remaining: 9.71s
813:	learn: 0.4100307	total: 42.3s	remaining: 9.66s
814:	learn: 0.4099833	total: 42.3s	remaining: 9.61s
815:	learn: 0.4099638	total: 42.4s	remaining: 9.56s
816:	learn: 0.4099486	total: 42.4s	remaining: 9.51s
817:	learn: 0.4099373	total: 42.5s	remaining: 9.45s
818:	learn: 0.4099139	total: 42.5s	remaining: 9.4s
819:	learn: 0.4098590	total: 42.6s	remaining: 9.35s
820:	learn: 0.4098335	total: 42.7s	remaining: 9.3s
821:	learn: 0.4098

961:	learn: 0.4061496	total: 51.9s	remaining: 2.05s
962:	learn: 0.4061288	total: 52s	remaining: 2s
963:	learn: 0.4060975	total: 52s	remaining: 1.94s
964:	learn: 0.4060600	total: 52.1s	remaining: 1.89s
965:	learn: 0.4060170	total: 52.2s	remaining: 1.84s
966:	learn: 0.4059948	total: 52.2s	remaining: 1.78s
967:	learn: 0.4059736	total: 52.3s	remaining: 1.73s
968:	learn: 0.4059436	total: 52.3s	remaining: 1.67s
969:	learn: 0.4059073	total: 52.4s	remaining: 1.62s
970:	learn: 0.4058730	total: 52.5s	remaining: 1.57s
971:	learn: 0.4058516	total: 52.6s	remaining: 1.51s
972:	learn: 0.4058463	total: 52.6s	remaining: 1.46s
973:	learn: 0.4058428	total: 52.7s	remaining: 1.41s
974:	learn: 0.4058229	total: 52.8s	remaining: 1.35s
975:	learn: 0.4058096	total: 52.8s	remaining: 1.3s
976:	learn: 0.4057693	total: 52.9s	remaining: 1.25s
977:	learn: 0.4057553	total: 52.9s	remaining: 1.19s
978:	learn: 0.4057544	total: 53s	remaining: 1.14s
979:	learn: 0.4057393	total: 53.1s	remaining: 1.08s
980:	learn: 0.4057099	

In [35]:
df_test = pd.read_csv("test.gz", compression='gzip', header='infer')
df_test_copy = df_test.copy()
unused_cols = ['id','site_id', 'app_id']
df_test_copy.drop(unused_cols, axis=1, inplace=True)
df_test_copy.drop(['hour', 'app_domain', 'app_category'], axis=1,inplace=True)

ctr_pred = opt_cat.predict_proba(df_test_copy)[:, 1]
all_id = df_test['id']
df_out = pd.DataFrame({'id': all_id, 'ctr': ctr_pred})
df_out.to_csv('Submission.csv', index=False)

### Logistic Regression (provided)

In [16]:
# embedding (all features are categorical)
print('embedding...')
from sklearn.feature_extraction import DictVectorizer
import pickle

try:
    with open('X_train_dict.pkl', 'rb') as ff:
        X_train_dict = pickle.load(ff)
    vectorizer = joblib.load('vectorizer.joblib')
    X_train = vectorizer.transform(X_train_dict)
    print('saved vectorizer loaded & applied to training set')
except:
    X_train_dict = list(df_copy.drop('click', axis=1).T.to_dict().values())
    with open('X_train_dict.pkl', 'wb') as ff:
        pickle.dump(X_train_dict, ff)
    vectorizer = DictVectorizer(sparse=True)
    X_train = vectorizer.fit_transform(X_train_dict) # can only see training dataset
    joblib.dump(vectorizer, 'vectorizer.joblib')
    print('imported data & built a vectorizer on the training set')

n, d = X_train.shape
print("n = {}, d = {}".format(n, d))

embedding...
imported data & built a vectorizer on the training set
n = 20000, d = 14076


In [17]:
X_train

<20000x14076 sparse matrix of type '<class 'numpy.float64'>'
	with 340000 stored elements in Compressed Sparse Row format>

In [18]:
# train
from sklearn.linear_model import LogisticRegression
print('fit a simple logistic regression with l1 regularization...')
clf = LogisticRegression(max_iter=20000, penalty='l1', solver='liblinear', C=1)
clf.fit(X_train, y_train)
print('...done training')

fit a simple logistic regression with l1 regularization...
...done training


In [20]:
##########################################################
######### testing code ###################################

# transform test data as well
print('loading and transforming test data...')
df_test = pd.read_csv("test.gz", compression='gzip', header='infer')
# df_test.set_index('id', inplace=True)
unused_cols = ['id', 'site_id', 'app_id']
df_test.drop(unused_cols, axis=1, inplace=True)

try:
    with open('X_test_dict.pkl', 'rb') as ff:
        X_test_dict = pickle.load('ff')
except:
    X_test_dict = list(df_test.T.to_dict().values())
    with open('X_test_dict.pkl', 'wb') as ff:
        pickle.dump(X_test_dict, ff)

X_test = vectorizer.transform(X_test_dict)

loading and transforming test data...


In [21]:
print('predicting and output to csv...')
ctr_pred = clf.predict_proba(X_test)[:, 1]
# save output: every line is (id, ctr_pred)
all_id = df_test['id']
df_out = pd.DataFrame({'id': all_id, 'ctr': ctr_pred})
df_out.to_csv('Submission.csv', index=False)

print('...done')

predicting and output to csv...
...done
